In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/segment/21_0_8019.png
/kaggle/input/segment/23_manual1.gif
/kaggle/input/segment/37_0_3779.png
/kaggle/input/segment/38_0_4273.png
/kaggle/input/segment/38_manual1.gif
/kaggle/input/segment/35_manual1.gif
/kaggle/input/segment/36_0_3768.png
/kaggle/input/segment/31_0_9189.png
/kaggle/input/segment/30_manual1.gif
/kaggle/input/segment/34_manual1.gif
/kaggle/input/segment/27_0_142.png
/kaggle/input/segment/27_manual1.gif
/kaggle/input/segment/29_manual1.gif
/kaggle/input/segment/29_0_478.png
/kaggle/input/segment/22_0_4498.png
/kaggle/input/segment/24_0_3480.png
/kaggle/input/segment/22_manual1.gif
/kaggle/input/segment/39_0_9807.png
/kaggle/input/segment/40_0_3246.png
/kaggle/input/segment/33_0_6318.png
/kaggle/input/segment/25_manual1.gif
/kaggle/input/segment/28_0_1954.png
/kaggle/input/segment/32_0_1886.png
/kaggle/input/segment/25_0_651.png
/kaggle/input/segment/30_0_3252.png
/kaggle/input/segment/33_manual1.gif
/kaggle/input/segment/39_manual1.gif
/kaggle/input/segmen

In [10]:
import os
import random
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau

def create_positive_pairs(dataset_path):
    pairs = []
    labels = []
    
    filenames = sorted(os.listdir(dataset_path))
    positives = []
    
    # Group the filenames into positive pairs
    for i in range(0, len(filenames), 2):
        image_path_1 = os.path.join(dataset_path, filenames[i])
        image_path_2 = os.path.join(dataset_path, filenames[i+1])
        
        if filenames[i][:2] == filenames[i+1][:2]:
            positives.append([image_path_1, image_path_2])
    
    # Check if there are enough positive pairs
    if len(positives) < 20:
        raise ValueError("Not enough positive pairs in the dataset.")
    
    # Shuffle the positive pairs
    random.shuffle(positives)
    
    # Take 20 positive pairs
    pairs.extend(positives[:20])
    labels.extend([1] * 20)  # Positive pair
    
    return pairs, labels

def create_negative_pairs(dataset_path):
    pairs = []
    labels = []
    
    filenames = sorted(os.listdir(dataset_path))
    negatives = []
    
    # Group the filenames into negative pairs
    for i in range(len(filenames)):
        for j in range(i+1, len(filenames)):
            image_path_1 = os.path.join(dataset_path, filenames[i])
            image_path_2 = os.path.join(dataset_path, filenames[j])
            
            if filenames[i][:2] != filenames[j][:2]:
                negatives.append([image_path_1, image_path_2])
    
    # Check if there are enough negative pairs
    if len(negatives) < 15:
        raise ValueError("Not enough negative pairs in the dataset.")
    
    # Shuffle the negative pairs
    random.shuffle(negatives)
    
    # Take 15 negative pairs
    pairs.extend(negatives[:15])
    labels.extend([0] * 15)  # Negative pair
    
    return pairs, labels

# Define the path to the dataset folder
dataset_path = '/kaggle/input/segment'

# Create positive pairs
positive_pairs, positive_labels = create_positive_pairs(dataset_path)

# Create negative pairs
negative_pairs, negative_labels = create_negative_pairs(dataset_path)

# Combine positive and negative pairs
pairs = positive_pairs + negative_pairs
labels = positive_labels + negative_labels

# Shuffle the pairs and labels
combined_data = list(zip(pairs, labels))
random.shuffle(combined_data)
pairs, labels = zip(*combined_data)

# Convert pairs and labels to numpy arrays
pairs = np.array(pairs)
labels = np.array(labels)

# Split into training and testing sets with a balanced distribution
train_ratio = 0.8
num_positive_pairs = len(positive_pairs)
num_negative_pairs = len(negative_pairs)

# Calculate the number of positive and negative pairs for training and testing
num_positive_train = int(train_ratio * num_positive_pairs)
num_positive_test = num_positive_pairs - num_positive_train
num_negative_train = int(train_ratio * num_negative_pairs)
num_negative_test = num_negative_pairs - num_negative_train

# Select the corresponding pairs and labels for training and testing
train_input_pairs = np.concatenate((pairs[:num_positive_train], pairs[num_positive_pairs:num_positive_pairs+num_negative_train]), axis=0)
train_labels = np.concatenate((labels[:num_positive_train], labels[num_positive_pairs:num_positive_pairs+num_negative_train]), axis=0)
test_input_pairs = np.concatenate((pairs[num_positive_train:num_positive_pairs], pairs[num_positive_pairs+num_negative_train:]), axis=0)
test_labels = np.concatenate((labels[num_positive_train:num_positive_pairs], labels[num_positive_pairs+num_negative_train:]), axis=0)

In [28]:
train_input_pairs.shape

(28, 2)

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras import backend as K

D0603 13:08:09.913040398      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0603 13:08:09.913071447      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0603 13:08:09.913074736      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0603 13:08:09.913077550      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0603 13:08:09.913079907      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0603 13:08:09.913082479      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0603 13:08:09.913084971      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0603 13:08:09.

In [4]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the input shape
input_shape = (256, 256, 3)

# Define the Siamese network architecture
def build_siamese_network(input_shape):
    input_image = Input(shape=input_shape)

    model = Conv2D(64, (10, 10), activation='relu')(input_image)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (7, 7), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (4, 4), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(256, (4, 4), activation='relu')(model)
    model = Flatten()(model)
    model = Dense(4096, activation='sigmoid')(model)

    encoded_image = model

    siamese_model = Model(input_image, encoded_image)
    return siamese_model

# Create the Siamese network model
siamese_model = build_siamese_network(input_shape)

# Compile the model with appropriate loss and optimizer
siamese_model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

# Generate some dummy training data
train_input_pairs = np.random.randn(28, 2, *input_shape)
test_input_pairs = np.random.randn(7, 2, *input_shape)
train_labels = np.random.randint(0, 2, size=(28,))
test_labels = np.random.randint(0, 2, size=(7,))

# Reshape the input pairs for training
train_input_pairs_reshaped = np.reshape(train_input_pairs, (train_input_pairs.shape[0], 2, *input_shape))
test_input_pairs_reshaped = np.reshape(test_input_pairs, (test_input_pairs.shape[0], 2, *input_shape))

# Define a custom distance function for comparing the encoded features
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_squared = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(sum_squared)

# Define the siamese network with the distance function
input_image_1 = Input(shape=input_shape)
input_image_2 = Input(shape=input_shape)

encoded_image_1 = siamese_model(input_image_1)
encoded_image_2 = siamese_model(input_image_2)

distance = Lambda(euclidean_distance)([encoded_image_1, encoded_image_2])

siamese_network = Model(inputs=[input_image_1, input_image_2], outputs=distance)

# Compile the siamese network
siamese_network.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

siamese_network.fit([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]], train_labels, 
                    validation_data=([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]], test_labels), 
                    batch_size=8, epochs=50)

# Evaluate the model on the testing data
result = siamese_network.evaluate([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]], test_labels)
loss = result[0]
accuracy = result[1]

print("Testing Loss: {:.4f}".format(loss))
print("Testing Accuracy: {:.2f}%".format(accuracy * 100))


Epoch 1/50
4/4 [==============================] - 15s 3s/step - loss: 0.7982 - val_loss: 0.7148
Epoch 2/50
4/4 [==============================] - 14s 3s/step - loss: 1.1270 - val_loss: 0.7363
Epoch 3/50
4/4 [==============================] - 13s 3s/step - loss: 0.9282 - val_loss: 0.6888
Epoch 4/50
4/4 [==============================] - 14s 3s/step - loss: 1.6221 - val_loss: 0.5693
Epoch 5/50
4/4 [==============================] - 14s 3s/step - loss: 1.2837 - val_loss: 1.0602
Epoch 6/50
4/4 [==============================] - 14s 3s/step - loss: 2.3266 - val_loss: 0.7293
Epoch 7/50
4/4 [==============================] - 14s 3s/step - loss: 1.0553 - val_loss: 0.9000
Epoch 8/50
4/4 [==============================] - 14s 3s/step - loss: 0.8891 - val_loss: 0.7926
Epoch 9/50
4/4 [==============================] - 14s 3s/step - loss: 1.7739 - val_loss: 0.8726
Epoch 10/50
4/4 [==============================] - 14s 3s/step - loss: 5.4599 - val_loss: 4.3043
Epoch 11/50
4/4 [======================

TypeError: 'float' object is not subscriptable

In [35]:
# Calculate training accuracy
train_pred = siamese_network.predict([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]])
train_pred = np.round(train_pred).flatten()
train_accuracy = np.mean(train_pred == train_labels)

# Calculate testing accuracy
test_pred = siamese_network.predict([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]])
test_pred = np.round(test_pred).flatten()
test_accuracy = np.mean(test_pred == test_labels)

print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy * 100))

1/1 [==============================] - 0s 225ms/step
Training Accuracy: 100.00%
Testing Accuracy: 57.14%


In [36]:
train_pred = siamese_network.predict([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]])
train_pred = np.round(train_pred).flatten()
train_accuracy = np.mean(train_pred == train_labels)

# Calculate testing accuracy
test_pred = siamese_network.predict([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]])
test_pred = np.round(test_pred).flatten()
test_accuracy = np.mean(test_pred == test_labels)

print("Training Accuracy: {:.2f}%".format(train_accuracy * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy * 100))

1/1 [==============================] - 0s 216ms/step
Training Accuracy: 100.00%
Testing Accuracy: 57.14%


In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the input shape
input_shape = (256, 256, 3)

# Define the Siamese network architecture
def build_siamese_network(input_shape):
    input_image = Input(shape=input_shape)

    model = Conv2D(64, (10, 10), activation='relu')(input_image)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (7, 7), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (4, 4), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(256, (4, 4), activation='relu')(model)
    model = Flatten()(model)
    model = Dense(4096, activation='sigmoid')(model)

    encoded_image = model

    siamese_model = Model(input_image, encoded_image)
    return siamese_model

# Create the Siamese network model
siamese_model = build_siamese_network(input_shape)

# Compile the model with appropriate loss and optimizer
siamese_model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

# Generate some dummy training data
train_input_pairs = np.random.randn(28, 2, *input_shape)
test_input_pairs = np.random.randn(7, 2, *input_shape)
train_labels = np.random.randint(0, 2, size=(28,))
test_labels = np.random.randint(0, 2, size=(7,))

# Reshape the input pairs for training
train_input_pairs_reshaped = np.reshape(train_input_pairs, (train_input_pairs.shape[0], 2, *input_shape))
test_input_pairs_reshaped = np.reshape(test_input_pairs, (test_input_pairs.shape[0], 2, *input_shape))

# Define a custom distance function for comparing the encoded features
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_squared = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(sum_squared)

# Define the siamese network with the distance function
input_image_1 = Input(shape=input_shape)
input_image_2 = Input(shape=input_shape)

encoded_image_1 = siamese_model(input_image_1)
encoded_image_2 = siamese_model(input_image_2)

distance = Lambda(euclidean_distance)([encoded_image_1, encoded_image_2])

siamese_network = Model(inputs=[input_image_1, input_image_2], outputs=distance)

# Compile the siamese network
siamese_network.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.0001))

siamese_network.fit([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]], train_labels, 
                    validation_data=([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]], test_labels), 
                    batch_size=32, epochs=50)




Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 1.3101 - val_loss: 0.8990
Epoch 2/50
1/1 [==============================] - 6s 6s/step - loss: 0.5419 - val_loss: 0.7676
Epoch 3/50
1/1 [==============================] - 6s 6s/step - loss: 0.6379 - val_loss: 0.6895
Epoch 4/50
1/1 [==============================] - 6s 6s/step - loss: 0.5232 - val_loss: 0.6627
Epoch 5/50
1/1 [==============================] - 6s 6s/step - loss: 0.4059 - val_loss: 0.6555
Epoch 6/50
1/1 [==============================] - 6s 6s/step - loss: 0.2793 - val_loss: 0.6576
Epoch 7/50
1/1 [==============================] - 6s 6s/step - loss: 0.1729 - val_loss: 0.6503
Epoch 8/50
1/1 [==============================] - 6s 6s/step - loss: 0.1160 - val_loss: 0.6415
Epoch 9/50
1/1 [==============================] - 6s 6s/step - loss: 0.0841 - val_loss: 0.6340
Epoch 10/50
1/1 [==============================] - 6s 6s/step - loss: 0.0843 - val_loss: 0.6272
Epoch 11/50
1/1 [==============================] 

In [39]:
train_pred1 = siamese_network.predict([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]])
train_pred1= np.round(train_pred1).flatten()
train_accuracy1 = np.mean(train_pred1 == train_labels)

# Calculate testing accuracy
test_pred1 = siamese_network.predict([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]])
test_pred1 = np.round(test_pred1).flatten()
test_accuracy1 = np.mean(test_pred1 == test_labels)

print("Training Accuracy: {:.2f}%".format(train_accuracy1 * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy1 * 100))

1/1 [==============================] - 0s 223ms/step
Training Accuracy: 100.00%
Testing Accuracy: 71.43%


In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the input shape
input_shape = (256, 256, 3)

# Define the Siamese network architecture
def build_siamese_network(input_shape):
    input_image = Input(shape=input_shape)

    model = Conv2D(64, (10, 10), activation='relu')(input_image)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (7, 7), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (4, 4), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(256, (4, 4), activation='relu')(model)
    model = Flatten()(model)
    model = Dense(4096, activation='sigmoid')(model)

    encoded_image = model

    siamese_model = Model(input_image, encoded_image)
    return siamese_model

# Create the Siamese network model
siamese_model = build_siamese_network(input_shape)

# Compile the model with appropriate loss and optimizer
siamese_model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

# Generate some dummy training data
train_input_pairs = np.random.randn(28, 2, *input_shape)
test_input_pairs = np.random.randn(7, 2, *input_shape)
train_labels = np.random.randint(0, 2, size=(28,))
test_labels = np.random.randint(0, 2, size=(7,))

# Reshape the input pairs for training
train_input_pairs_reshaped = np.reshape(train_input_pairs, (train_input_pairs.shape[0], 2, *input_shape))
test_input_pairs_reshaped = np.reshape(test_input_pairs, (test_input_pairs.shape[0], 2, *input_shape))

# Define a custom distance function for comparing the encoded features
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_squared = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(sum_squared)

# Define the siamese network with the distance function
input_image_1 = Input(shape=input_shape)
input_image_2 = Input(shape=input_shape)

encoded_image_1 = siamese_model(input_image_1)
encoded_image_2 = siamese_model(input_image_2)

distance = Lambda(euclidean_distance)([encoded_image_1, encoded_image_2])

siamese_network = Model(inputs=[input_image_1, input_image_2], outputs=distance)

# Compile the siamese network
siamese_network.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

siamese_network.fit([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]], train_labels, 
                    validation_data=([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]], test_labels), 
                    batch_size=32, epochs=50)




Epoch 1/50
1/1 [==============================] - 8s 8s/step - loss: 5.9909 - val_loss: 4.8223
Epoch 2/50
1/1 [==============================] - 6s 6s/step - loss: 1.6324 - val_loss: 1.0089
Epoch 3/50
1/1 [==============================] - 6s 6s/step - loss: 2.5980 - val_loss: 1.8556
Epoch 4/50
1/1 [==============================] - 6s 6s/step - loss: 2.0769 - val_loss: 6.0597
Epoch 5/50
1/1 [==============================] - 6s 6s/step - loss: 8.1050 - val_loss: 6.6107
Epoch 6/50
1/1 [==============================] - 6s 6s/step - loss: 9.3651 - val_loss: 6.6107
Epoch 7/50
1/1 [==============================] - 6s 6s/step - loss: 9.3651 - val_loss: 6.6107
Epoch 8/50
1/1 [==============================] - 6s 6s/step - loss: 9.3651 - val_loss: 6.6107
Epoch 9/50
1/1 [==============================] - 6s 6s/step - loss: 9.3651 - val_loss: 6.6107
Epoch 10/50
1/1 [==============================] - 6s 6s/step - loss: 9.3651 - val_loss: 6.6107
Epoch 11/50
1/1 [==============================] 

In [41]:
train_pred2 = siamese_network.predict([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]])
train_pred2= np.round(train_pred1).flatten()
train_accuracy2 = np.mean(train_pred2 == train_labels)

# Calculate testing accuracy
test_pred2 = siamese_network.predict([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]])
test_pred2 = np.round(test_pred1).flatten()
test_accuracy2 = np.mean(test_pred2 == test_labels)

print("Training Accuracy: {:.2f}%".format(train_accuracy2 * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy2 * 100))

1/1 [==============================] - 0s 199ms/step
Training Accuracy: 53.57%
Testing Accuracy: 57.14%


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define the input shape
input_shape = (256, 256, 3)

# Define the Siamese network architecture
def build_siamese_network(input_shape):
    input_image = Input(shape=input_shape)

    model = Conv2D(64, (10, 10), activation='relu')(input_image)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (7, 7), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(128, (4, 4), activation='relu')(model)
    model = MaxPooling2D()(model)
    model = Conv2D(256, (4, 4), activation='relu')(model)
    model = Flatten()(model)
    model = Dense(4096, activation='sigmoid')(model)

    encoded_image = model

    siamese_model = Model(input_image, encoded_image)
    return siamese_model

# Create the Siamese network model
siamese_model = build_siamese_network(input_shape)

# Compile the model with appropriate loss and optimizer
siamese_model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.001))

# Generate some dummy training data
train_input_pairs = np.random.randn(28, 2, *input_shape)
test_input_pairs = np.random.randn(7, 2, *input_shape)
train_labels = np.random.randint(0, 2, size=(28,))
test_labels = np.random.randint(0, 2, size=(7,))

# Reshape the input pairs for training
train_input_pairs_reshaped = np.reshape(train_input_pairs, (train_input_pairs.shape[0], 2, *input_shape))
test_input_pairs_reshaped = np.reshape(test_input_pairs, (test_input_pairs.shape[0], 2, *input_shape))

# Define a custom distance function for comparing the encoded features
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_squared = tf.reduce_sum(tf.square(vector1 - vector2), axis=1, keepdims=True)
    return tf.sqrt(sum_squared)

# Define the siamese network with the distance function
input_image_1 = Input(shape=input_shape)
input_image_2 = Input(shape=input_shape)

encoded_image_1 = siamese_model(input_image_1)
encoded_image_2 = siamese_model(input_image_2)

distance = Lambda(euclidean_distance)([encoded_image_1, encoded_image_2])

siamese_network = Model(inputs=[input_image_1, input_image_2], outputs=distance)

# Compile the siamese network
siamese_network.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=0.0001))

siamese_network.fit([train_input_pairs_reshaped[:, 0], train_input_pairs_reshaped[:, 1]], train_labels, 
                    validation_data=([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]], test_labels), 
                    batch_size=32, epochs=80)




Epoch 1/80
1/1 [==============================] - 9s 9s/step - loss: 1.5948 - val_loss: 0.9890
Epoch 2/80
1/1 [==============================] - 6s 6s/step - loss: 0.5826 - val_loss: 0.8248
Epoch 3/80
1/1 [==============================] - 6s 6s/step - loss: 0.8625 - val_loss: 0.7274
Epoch 4/80
1/1 [==============================] - 6s 6s/step - loss: 0.6759 - val_loss: 0.6728
Epoch 5/80
1/1 [==============================] - 6s 6s/step - loss: 0.5427 - val_loss: 0.6412
Epoch 6/80
1/1 [==============================] - 6s 6s/step - loss: 0.4340 - val_loss: 0.6251
Epoch 7/80
1/1 [==============================] - 6s 6s/step - loss: 0.3330 - val_loss: 0.6173
Epoch 8/80
1/1 [==============================] - 6s 6s/step - loss: 0.2289 - val_loss: 0.6148
Epoch 9/80
1/1 [==============================] - 6s 6s/step - loss: 0.1236 - val_loss: 0.6171
Epoch 10/80
1/1 [==============================] - 6s 6s/step - loss: 0.0932 - val_loss: 0.6155
Epoch 11/80
1/1 [==============================] 

In [1]:
from sklearn.metrics import confusion_matrix

# Generate predictions for the testing data
test_pred1 = siamese_network.predict([test_input_pairs_reshaped[:, 0], test_input_pairs_reshaped[:, 1]])
test_pred1 = np.round(test_pred1).flatten()

# Create the confusion matrix
cm = confusion_matrix(test_labels, test_pred1)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

NameError: name 'siamese_network' is not defined